# Load libraries and Data
---




In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
import pandas as pd
data_train = pd.read_json('/content/drive/My Drive/Master thesis/Datasets/Arabic_hatespeech-master-nuha/Hate_Speech/train.json', lines=True)
data_test = pd.read_json('/content/drive/My Drive/Master thesis/Datasets/Arabic_hatespeech-master-nuha/Hate_Speech/test.json', lines=True)

                  
df_train = pd.DataFrame(data_train,columns=['full_text','hate'])
df_test = pd.DataFrame(data_test,columns=['full_text'])

df_train
# data_json.head()

,full_text,hate
0,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...,no
1,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,no
2,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه,yes
3,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,yes
4,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,yes
...,...,...
5564,"🔺محاضرة للإخوة الألبانيين بعنوان ""عقيدة أهل ال...",no
5565,الدورة الحضارية قد دارت وهذا زمان انحدار أهل ا...,no
5566,New post (تعرف على قصة حسن المندلاوي مع خواته ...,no
5567,في فرق بين السنّي والوهابي \nوشكراً,no


In [50]:
print(df_train.shape)
print(df_test.shape)
# print(df.info)
# print(df.describe)

(5569, 2)
(567, 1)


# Split data 

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train['full_text'].values, df_train['hate'].values, test_size=0.1)

# Data Cleaning
---








In [53]:
# initialize Tokenizer to encode strings into integers
tokenizer = Tokenizer()

# calculate number of rows in our dataset
num_rows = df_train.shape[0]

# create vocabulary from all words in our dataset for encoding
tokenizer.fit_on_texts(df_train['full_text'].values)

# max length of 1 row (number of words)
row_max_length = max([len(x.split()) for x in df_train['full_text'].values])

# count number of unique words
vocabulary_size = len(tokenizer.word_index) + 1

# convert words into integers
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)
X_sub_tokens = tokenizer.texts_to_sequences(df_test['full_text'].values)

# ensure every row has same size - pad missing with zeros
X_train_pad = pad_sequences(X_train_tokens, maxlen=row_max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=row_max_length, padding='post')
X_sub_pad = pad_sequences(X_sub_tokens, maxlen=row_max_length, padding='post')

In [63]:
EMBEDDING_DIM = 256

model = Sequential()
model.add(Embedding(vocabulary_size, EMBEDDING_DIM, input_length=row_max_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(128)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2)) 
model.add(Dense(1,name='out_layer'))
# model.add(Dense(target_length, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)

# model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          # validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
history = model.fit(X_train_pad, y_train, epochs=5, validation_data=0.2, batch_size=128, callbacks=[callback])

Epoch 1/5


UnimplementedError: ignored

# Data Cleaning
---








1.   Remove punctuation marks by using Regex.


In [ ]:
import re,string

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', str(text))
    return text_nopunct
data['Text_Without_punct'] = data['text'].apply(lambda x: remove_punct(x))

2.   tokenize the comments by using NLTK’s word_tokenize


In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in data.Text_Without_punct]


3.  lower case the data



In [ ]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

4. Remove stop words from data using NLTK’s stopwords.

In [ ]:
 nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('arabic')
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]
data['Text_Final'] = [' '.join(sen) for sen in filtered_words]
data['tokens'] = filtered_words

In [ ]:
data

# Splitting Data into Test and Train


In [ ]:
#Now we split our data set into train and test. We will use 90 % data for training and 10 % for testing.
!pip install scikit-learn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = data.text.astype(str)
Y = data.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
#data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

# Process the data¶

*   Tokenize the data and convert the text to sequences.
*   Add padding to ensure that all the sequences have the same shape.
*   There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.






In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
import numpy as np

max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

# RNN

In [ ]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.optimizers import RMSprop

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
#Call the function and compile the model.
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
#Fit on the training data.
from keras.callbacks import EarlyStopping

model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,y_test)


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))